**Содержание**<a id='toc0_'></a>    
- [Контекстуализированные представления и перенос знаний](#toc1_)    
    - [Основная идея transfer learning](#toc1_1_1_)    
    - [Существует несколько способов работы с предобученной моделью](#toc1_1_2_)    
  - [OpenAI трансформер](#toc1_2_)    
  - [BERT](#toc1_3_)    
  - [ Embeddings from Language Models, ELMo](#toc1_4_)    
  - [Итого](#toc1_5_)    
- [Дальнейшее развитие области обработки текста. Transformer-XL](#toc2_)    
    - [Рекуррентный механизм на основе сегментов](#toc2_1_1_)    
    - [Относительное позиционное кодирование](#toc2_1_2_)    
    - [Преимущества Transformer-XL](#toc2_1_3_)    
- [Generative Pre-trained Transformer, GPT-2](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Контекстуализированные представления и перенос знаний](#toc0_)

**Дальше - некоторый обзор актуальных архитектур без особой конкретики**

В сфере NLP 2018 год стал достаточно переломным моментом. 
1. К этому периоду достаточно продвинулось концептуальное понимание того, как лучше всего представлять слова и предложения (чтобы учитывать их семантику, какой-то лингвистический смысл). 
2. Были предложены мощные и выложенные с уже предобученными весами, модели, которые можно было скачать и использовать в своих прикладных задачах. И эти модели работали очень и очень хорошо. 
 
Этот момент был назван "моментом и ImageNet в NLP"[1] — со ссылкой на то, как несколько лет назад подобные разработки в [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning) ускорили развитие компьютерного зрения.

[1] [NLP's ImageNet moment has arrived](https://ruder.io/nlp-imagenet/), 12 July 2018

### <a id='toc1_1_1_'></a>[Основная идея transfer learning](#toc0_)

Суть в том, что сначала модель предобучается (или берется готовая) на большом корпусе, а затем для решения конкретной задачи на сравнительно небольшом размеченном корпусе делается [файнтюнинг](https://en.wikipedia.org/wiki/Fine-tuning) (fine-tune) - уточняются предобученные веса сети в контексте решения интересующей конкретной задачи.

В **NLP** часто в качестве transfer learning применяется следующая схема: 
- на этапе предобучения модель пытается предсказывать пропущенные слова в тексте (то есть, решается задача [языкового моделирования](https://en.wikipedia.org/wiki/Language_model))
  - для решения этой задачи нам не нужны размеченные данные, достаточно поочерёдно заменять некоторой маской слова и пытаться их предсказывать
  - в процессе такого обучения модель приобретёт знания о том, как устроен язык (запомнит, какие слова встречаются рядом с другими словами, возможно, запомнит более длинные паттерны)
- на этапе дообучения полученные веса модели файнтюнятся с использованием размеченных данных и модель учится решать некоторую задачу. 

Одно из основных преимуществ предобученных моделей состоит в том, что мы можем использовать значительно меньше размеченных данных для дообучения, чем при тренировке модели с нуля

### <a id='toc1_1_2_'></a>[Существует несколько способов работы с предобученной моделью](#toc0_)

1. Не менять веса предобученной сети (т.е. заняться извлечение признаков из готовой сети **feature extraction**). Например, мы можем добавить "наверх" предобученной модели линейный слой и обучить только его, не меняя веса предобученной сети (в таких случаях говорят, что веса предобученной сети "заморожены").
2. Менять веса предобученной сети (**fine-tuning**). В таком случае веса предобученной сети используются в качестве инициализации для downstream модели. Менять веса предобученной сети можно по-разному:
- Заморозить все слои предобученной сети и обучить только верхние "надстроенные" слои на решение конкретной задачи (так это ж первый случай, не?).
- Заморозить все слои и размораживать их постепенно.
  - Основная интуиция заключается в том, что одновременное обучение всех слоев сети сразу на данных другого домена может привести к нестабильности. Вместо этого мы обучаем слои индивидуально, чтобы дать им время адаптироваться к новой задаче и/или данным.
- Не замораживать слои предобученной сети, но использовать различные (небольшие) learning rates для обучения разных слоев. Мы хотим использовать более маленькие значения learning rate, чтобы избежать перезаписи полезной информации. Более низкий learning rate особенно важен при обучении нижних слоев сети (поскольку они собирают более общую информацию), на ранних этапах обучения (поскольку модели все еще необходимо адаптироваться к целевому распределению) и на поздних этапах обучения (когда модель близка к тому, чтобы сойтись). Чтобы поддерживать более низкие скорости обучения на ранних этапах обучения, можно использовать треугольный график изменения learning rate (triangular learning rate schedule),  который также называют "прогревом" скорости обучения (learning rate warm-up).
- Не замораживать слои, но использовать регуляризацию (штраф за большие веса: $L_1$ - по модулю, $L_2$ - по квадрату) для того, чтобы веса оставались близкими к весам предобученной сети.

## <a id='toc1_2_'></a>[OpenAI трансформер](#toc0_)

Июнь 2018 года, OpenAI

Мы хотим предобучить языковую модель на большом неразмеченном корпусе, так, чтобы дальше её можно было адаптировать для решения конкретных других задач, поэтому:
- для реализации идеи transfer learning совершенно не нужна вся архитектура трансформера, а достаточно ограничиться только декодером

**Архитектура OpenAI трансформера:**

Состоит из 12 слоев **декодеров** [ванильного](https://en.wikipedia.org/wiki/Vanilla_software) трансформера
  - декодер в трансформере - это self-attention модуль + максирование токенов справа
    - в ванильном трансформере их 6

Полученную архитектуру учили решать задачу языкового моделирования: предсказывать следующее слово при известном контексте, где как раз и пригодилась особенность декодера с маскировкой токенов, которые справа.

<img src="./img/transformer-decoder-block-2.png" width="400">

**Трансфер лернинг:**

Трансформер обучен на текстах - т.е. на каждый токенизированный вход он дает некоторый, одному ему известный выход - вещественный вектор. Дальше свои +/- соображения как это может выглядеть

1. Если к нему сверху прикрутить полносвязный слой с софтмаксом, то это будет задача **софтмакс-регрессии** (многоклассовое обобщение логистической регрессии), которая, оказывается, дает хорошие результаты (предикторы - то что насчитал по тексту тарансформер, зависимая переменная - некоторый класс (например, спам/не спам или большее количество классов)). И для этого достаточно дообучить прикрученный к трансформеру слой на небольшом размеченном корпусе.

2. Хорошо решается задача оценки смысловой близости предложений - в трансформер поступает (текст1 + текст2), потом (текст2 + текст1) - каждая последовательность преобразуется трансформером в тензоры признаков (где каждый токен - вектор эмбеддингов в контексте других токенов предложения). Полученные тензоры, например, умножаются (у них жи один эмбеддинг) и результат поступает на полносвязный слой (который в данном случае даже и обучать не надо получается(?)) для классификации (оценки близости тензоров).
   
3. Пхожим образом с п.2 решается задача выбора ответа на вопрос - в трансформер подаются по очереди (вопрос + ответ1), (вопрос + ответ2), ..., (вопрос + ответN), каждый результат последовательно подается на полносвязный слой, каждый из которых даст оценку


[1] Peters M. E. et al. Deep contextualized word representations //arXiv preprint arXiv:1802.05365. – 2018.  
[2] Akbik A., Blythe D., Vollgraf R. Contextual string embeddings for sequence labeling //Proceedings of the 27th International Conference on Computational Linguistics. – 2018. – С. 1638-1649.  
[3] Baevski A. et al. Cloze-driven pretraining of self-attention networks //arXiv preprint arXiv:1903.07785. – 2019.  
[4] The State of Transfer Learning in NLP, 18 August 2019  

*А что — если попробовать учиться предсказывать не только следующее (а, может быть, даже и предыдущее слово), а пойти ещё дальше? Например, заменять маской произвольное слово и предсказывать это произвольное слово внутри предложения*
 

## <a id='toc1_3_'></a>[BERT](#toc0_)

Относится к моделям со свободным порядком факторизации. "**Bi-directional Encoder Representations from Transformers**", то есть — двунаправленное представление на основе трансформеров.

**Архитектура:**

Состоит из 12 ($BERT_{base}$) или 24 ($BERT_{large}$) слоев **энкодеров** [ванильного](https://en.wikipedia.org/wiki/Vanilla_software) (не совсем) трансформера
  - энкодер в трансформере - это self-attention модуль:
    - 12 голов в multi-headed attention (16 для $BERT_{large}$) 
      - в стандартном ванильном трансформере их было всего 8
    - размерность входа и выходного полносвязного слоя энкодера, а **также эмбеддинга** принята равной (и кратной числу голов)
      - 768 или 1024 ($BERT_{large}$), в ванильном трансформере,соответственно, было 512

<img src="./img/transformer-encoder-block-2.png" width="400">

Гиперпараметрами модели являются H — размерность скрытого пространства кодировщика, L — количество слоёв-кодировщиков, A — количество голов в механизме внимания.

**Контуры алгоритма обучения:**

На вход BERT принимает последовательность токенов, которые проходят через весь стэк. 

На каждом слое работает энкодер self-attention (включая выходной полносвязный слой) и, далее, пв следующий encoder. 

Пока вроде никаких принципиальных отличий BERT от OpenAI трансформера нет, но это не так.

Основная фишка BERT — в его предобучении. В процессе предобучения BERT тренируется решать не одну задачу языкового моделирования, а целых две задачи: 
  - маскированное языковое моделирование (BERT маскирует 15% входных токенов и пытается их предсказать), т.е. в отличие от OpenAI трансформера BERT предсказывает не следующий токен, а любой токен в любом произвольном месте предложения
  -  вход подаются также пары предложений (A и B) и нужно будет определить, является ли B логичным продолжением A

[1] Devlin J. et al. Bert: Pre-training of deep bidirectional transformers for language understanding //arXiv preprint arXiv:1810.04805. – 2018.

**Формат данных BERT**

Для того, чтобы удобно работать с обеими задачами, BERT нужен особенный формат входных данных.

1. Предложения начинаются с метки "CLS"
   - на выходе напротив этой метки (общей для всех текстов, поэтому вот) соберется вектор эмбеддингов, который будет характеризовать текст в целом
2. Для второй задачи, когда на вход два предложения, то токены первого предложения отделим от токенов второго меткой "SEP"

Для токенизации текста в BERT используется аналог BPE - wordpieces. Этот алгоритм на этапе обучения выявляет устойчивые n-граммы символов, а при предсказании разбивает текст на наиболее подходящую последовательность этих n-грамм. Каждую n-грамму можно представить как числом (её номером в словаре), так и последовательностью символов.

**Трансфер лернинг с BERT**

<img src="./img/bert2.png" width="700">

Варианты:
1. В статье про BERT предлагается, в качестве классификатора на этапе дообучения, использовать всего один линейный слой с софтмаксом. 

2. Можно научить BERT работать с вопросно-ответным поиском или переводить предложения с французского на английский, либо на любой другой язык.

3. Векторы, которые нам выдаёт предобученный BERT, можно сами по себе **использовать в качестве эмбеддингов** (контекстуализированные эмбеддинги). Более того:
    - можно использовать векторы с последнего слоя (логично)
    - можно взять сумму по всем слоям
    - всякие другие комбинации
      - например, было показано, что в задаче извлечения именованных сущностей лучше всего показала конкатенация эмбеддингов с последних четырех слоев в длинный эмбеддинг
      - такой подход хорошо справляется с **частеречной омонимией** ("Ворота замка закрыты на замок") и одинаковые токены получают разные, но сопоставимые, эмбеддинги в разном контексте

[1] http://jalammar.github.io/illustrated-transformer/  
[2] http://jalammar.github.io/illustrated-bert/

## <a id='toc1_4_'></a>[ Embeddings from Language Models, ELMo](#toc0_)

В основе модели лежит двунаправленная LSTM. Модель выдает качественные контекстуализированные эмбеддинги.

ELMo смотрит на всё предложение, прежде чем присвоить слову какое-то векторное представление, и, точно так же, как и ранее рассмотренные модели, учиться решать задачу языкового моделирования, но уже не с помощью архитектуры на основе трансформера, а с помощью LSTM (а точнее, даже двух LSTM, которые смотрят в разные стороны).

$$ P(t_1, t_2, ..., t_N) = \prod_{k=1}^N P(t_k|t_{1},..,t_{k-1}) \\
P(t_1, t_2, ..., t_N) = \prod_{k=1}^N P(t_k|t_{k+1},..,t_{N} )$$

Итоговое векторное представление слова получается путём комбинации скрытых состояний из обеих частей LSTM. 

Можно объединять эти скрытые состояния разными способами. Например, вектора скрытых состояний можно домножить на веса и, затем, сконкатенировать (или даже суммировать) в один вектор. 

Выбор варианта комбинации скрытых слоёв двух частей LSTM, скорее, относится к инженерным задачам, некоторым инженерным хакам, подбору эвристик. В любом случае, в таком векторе будет информация о всей последовательности, представленная лучше или хуже, но будет.

[1] Peters M. E. et al. Deep contextualized word representations //arXiv preprint arXiv:1802.05365. – 2018.

## <a id='toc1_5_'></a>[Итого](#toc0_)
OpenAI Transformer
- Однонаправленное языковое моделирование

BERT
- "Маскированное" языковое моделирование и предсказание, является ли второе предложение логическим продолжением предыдущего

ELMO
- Двунаправленное языковое моделирование: предсказание следующего и предыдущего токена по соответствующему контексту (токены левее или правее предсказываемого токена соответственно)

# <a id='toc2_'></a>[Дальнейшее развитие области обработки текста. Transformer-XL](#toc0_)

На базе Transformer-XL построена модель **XLNet**, которая во многих задачах показала более высокие метрики, чем BERT (правда BERT допилили и он опять почти в топе в виде RoBERTa, ALBERTa, DeBERTa)).

Ограничения ванильного трансформера[4]:
- контекст фиксированной длины (неучет более длинных зависимостей),
- проблема фрагментации контекста (попадает в разные куски)

Решение:
- Рекуррентный механизм на уровне сегментов (segment level recurrenct mechanism)
- \+ Специальное относительное позиционное кодирование (relative position encoding)

Подход предложил Гугл и назвал его Transformer-XL[3]

### <a id='toc2_1_1_'></a>[Рекуррентный механизм на основе сегментов](#toc0_)

<img src="./img/XL.png">

Во время обучения, представления, вычисленные для предыдущего сегмента, будут фиксироваться и кэшироваться для повторного использования, и этот расширенный контекст модель будет учитывать, когда она обрабатывает следующий сегмент. 

Это дополнительное соединение увеличивает максимально возможную длину зависимости в N раз, где N — это глубина сети. 

И, поскольку теперь контекстная информация может перетекать через границы сегмента, это обеспечивает хорошее качество работы трансформера.

Кроме того, этот механизм повторения также решает проблему фрагментации контекста, обеспечивая необходимый контекст для токенов в передней части нового сегмента. Звучит очень круто, и если мы посмотрим на схему, то можно убедиться, что теперь у долговременных зависимостей есть гораздо больше шансов не потеряться и сыграть свою роль в процессе обучения. 

**Проблема:**
Позиционные кодировки не являются когерентными при повторном использовании предыдущих сегментов[2]. Эт, короче про то, что мол как ты отразишь в эмбеддинге позицию токена в предыдущем сегменте относительно позиции в текущем сегменте?

### <a id='toc2_1_2_'></a>[Относительное позиционное кодирование](#toc0_)

Пусть старый сегмент у нас с контекстными позициями: "0", "1", "2", "3". Когда новый сегмент обрабатывается, у нас есть позиции "0", "1", "2", "3", "0", "1", "2", "3" — для двух объединённых сегментов, где семантика каждого идентификатора позиции не работает для этой объединенной последовательности.

Алгоритм приводится в [3]. Там без примера не разберешься, но суть в том, что предложенная формула для расчета внимания состоит из локальной и глобальной части (слагаемые), где в глобальной части позиции токенов кодируются по всему тексту матрицей синусных сигналов, в локальной - только по сегменту.

*Тупой вопрос возник, а че просто цифорку - номер предложения/фрагмента/сегмента не добавлять к эмбеддингу? А нужно относительное? Поделить на число предложений/фрагментов/сегментов?*

### <a id='toc2_1_3_'></a>[Преимущества Transformer-XL](#toc0_)

Быстрый, т.к. много нужного кешируется

При одинаковой глубине охват контекста на 80% больше RNN и на 450% больше ванильного трансформера
- лучше работает на длинных последовательностях

[1] Mikolov T. et al. Distributed representations of words and phrases and their compositionality //Advances in neural information processing systems. – 2013. – С. 3111-3119.  
[2] Transformer-XL: Unleashing the Potential of Attention Models, January 29, 2019, Zhilin Yang and Quoc Le, Google AI  
[3] Dai Z. et al. Transformer-xl: Attentive language models beyond a fixed-length context //arXiv preprint arXiv:1901.02860. – 2019.  
[4] Young T. et al. Recent trends in deep learning based natural language processing //ieee Computational intelligenCe magazine. – 2018. – Т. 13. – №. 3. – С. 55-75.  
[5] Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly learning to align and translate. arXiv preprint arXiv:1409.0473 (2014).

# <a id='toc3_'></a>[Generative Pre-trained Transformer, GPT-2](#toc0_)

Авторегрессионная генеративная языковая модель на архитектуре трансформер (на **декодере**). Преемник сети под названием GPT, которая была создана сотрудниками OpenAI. В 2020 году выпустили GPT-3

Модель GPT-2 была обучена на задаче языкового моделирования (училась предсказывать следующее слово), но обучалась на очень большом корпусе интеренет текста, 40Гб, 1,5 млрд.параметров, размерность входа 1024 токена. 
- Корпус для GPT-3 уже 540 Гб, 175 млрд. параметров размерность входа 2048 токенов.

<img src="./img/gpt2-token-embeddings-wte-2.png" width="400">
<img src="./img/gpt2-positional-encoding.png" width="400">

Разработчки даже решили поднять интерес к модели и начали панику по поводу какая она слишком замечательная и как она щас заполнит интернет фейками. Поэтому они специально скукожили ее веса перед релизом. Сгенерированные статьи, которые прошли в рецензируемые журналы, чат-боты, советующие человекам самовыпилиться - это все она.
- критики, однако считают, что "построить интеллектуальные машины путём масштабирования языковых моделей — всё равно что строить высотные самолёты для полёта на Луну", просто не та технология в принципе

Модель может выучить принципы работы вопросно-ответных систем или машинного перевода, или систем для суммаризации текстов прямо из сырых текстов, пока она учится решать задачу языкового моделирования на сырых данных. То есть, иногда модель может работать достаточно неплохо даже без файнтюнинга. 

В статье про GPT-2 были показаны хорошие результаты для zero-shot learning[4] (выделение ранее неизвестных классов) только для одной задачи из восьми. Для остальных задач результаты работы модели без дообучения, хоть и не ужасные, но всё ещё далеки от идеальных. Задача, на которой zero-shot learning[4] для GPT-2 всё-таки сработал — это задача понимания прочитанного текста (или "reading comprehension[3]").

Датасет для этой задачи выглядел следующим образом — он состоял из пар "документ и диалог про этот документ". Документы относились к семи разным доменам и сетке нужно было научиться отвечать на вопросы по этому документу. Тем не менее, было отмечено, что, хоть GPT-2 и показал отличные результаты, он часто выучивал достаточно простые эвристики (например, отвечать на вопрос "кто" единственно встретившимся в тексте именем). 

**Интересно:**
[MusicTransformer](https://magenta.tensorflow.org/music-transformer) имеет похожую автрегрессионную архитектуру на основе декодера трансорформера

[1] https://openai.com/blog/unsupervised-sentiment-neuron/  
[2] https://github.com/eukaryote31/openwebtext  
[3] https://paperswithcode.com/task/reading-comprehension  
[4] https://paperswithcode.com/task/zero-shot-learning

<span><h2>Что еще почитать</h2>

<p>Список ссылок на статьи про архитектуры, рассмотренные в лекции:</p>

<ol>
	<li>OpenAI Transformer:&nbsp;<a href="https://openai.com/blog/language-unsupervised/" rel="noopener noreferrer nofollow" target="_blank">Improving Language Understanding by Generative Pre-Training</a></li>
	<li>ELMO:&nbsp;<a href="https://arxiv.org/abs/1802.05365" rel="noopener noreferrer nofollow" target="_blank">Deep contextualized word representations</a></li>
	<li>BERT:&nbsp;<a href="https://arxiv.org/abs/1810.04805" rel="noopener noreferrer nofollow" target="_blank">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding</a></li>
	<li>Transformer-XL:&nbsp;<a href="https://arxiv.org/abs/1901.02860" rel="noopener noreferrer nofollow" target="_blank">Transformer-XL: Attentive Language Models Beyond a Fixed-Length Context</a></li>
	<li>GPT-2:&nbsp;<a href="https://openai.com/blog/better-language-models/" rel="noopener noreferrer nofollow" target="_blank">Language Models are Unsupervised Multitask Learners</a></li>
</ol>

<p>Серия статей от&nbsp;<a href="http://jalammar.github.io/" rel="noopener noreferrer nofollow" target="_blank">Jay Alammar</a>&nbsp;(отличные посты с очень понятными картинками):</p>

<ol>
	<li><a href="http://jalammar.github.io/illustrated-transformer/" rel="noopener noreferrer nofollow" target="_blank">Illustrated Transformer</a></li>
	<li><a href="http://jalammar.github.io/illustrated-bert/" rel="noopener noreferrer nofollow" target="_blank">Illustrated BERT</a>&nbsp;(материалы отсюда были использованы в лекции)</li>
	<li><a href="http://jalammar.github.io/illustrated-gpt2/" rel="noopener noreferrer nofollow" target="_blank">Illustrated GPT-2</a></li>
</ol>

<p>Еще немного блог-постов:</p>

<ol>
	<li><a href="https://allennlp.org/elmo" rel="noopener noreferrer nofollow" target="_blank">ELMO от AllenNLP</a></li>
	<li><a href="https://ai.googleblog.com/2019/01/transformer-xl-unleashing-potential-of.html" rel="noopener noreferrer nofollow" target="_blank">Transformer-XL от Google AI</a></li>
	<li><a href="https://medium.com/dair-ai/a-light-introduction-to-transformer-xl-be5737feb13" rel="noopener noreferrer nofollow" target="_blank">Простая статья на Medium про Transformer-XL</a></li>
</ol>

<p>Ссылки на код моделей:</p>

<ol>
	<li><a href="https://github.com/openai/finetune-transformer-lm" rel="noopener noreferrer nofollow" target="_blank">OpenAI Transformer</a></li>
	<li><a href="https://github.com/allenai/bilm-tf" rel="noopener noreferrer nofollow" target="_blank">ELMO (Tensorflow)</a></li>
	<li><a href="https://github.com/google-research/bert" rel="noopener noreferrer nofollow" target="_blank">BERT (Tensorflow)</a></li>
	<li><a href="https://github.com/kimiyoung/transformer-xl/" rel="noopener noreferrer nofollow" target="_blank">Transformer-XL</a></li>
	<li><a href="https://github.com/openai/gpt-2" rel="noopener noreferrer nofollow" target="_blank">GPT-2</a></li>
</ol></span>